# PLSE ETL - Part 2: Scraping PDFs to DB
Note: Using Sample Data Set (count=199)

In [ ]:
from PyPDF2 import PdfFileReader

In [ ]:
from os import listdir
from os.path import isfile, join

In [ ]:
import re

In [ ]:
path_to_dockets = '../tmp/sample_dockets'

In [ ]:
dockets = [f for f in listdir(path_to_dockets) if isfile(join(path_to_dockets, f))]

In [ ]:
def compiled_pattern(pattern):
    return re.compile(r'{}'.format(pattern))

In [ ]:
dn = '(?<=Docket\sNumber\:\s)\w{2}\-\d{2}\-\w{2}\-\d{7}\-\d{4}'
dn_t = ['DOCKETDocket Number: MC-51-CR-0129461-1989Court Case']

In [ ]:
fd = '(?<=Date\sFiled\:\s{2})\d{2}\/\d{2}\/\d{4}'
fd_t = ['1989Date Filed:  01/25/1989Judge']

In [ ]:
ja = '(?<=Judge\sAssigned\n\:\s{2})\w+\,\s\w+(\s\w\.)?(?=Initiation\sDate\:)'
ja_t = ['01/25/1989Judge Assigned\n:  Kafrissen, Arthur S.Initiation Date:', '01/25/1989Judge Assigned\n:  Kafrissen, ArthurInitiation Date:']

In [ ]:
fml = '(?<=Pennsylvaniav\.)(\w|\s)+(?=CASE)'
fml_t = ['Commonwealth of Pennsylvaniav.Carl R BurchCASE INFORMATION', 'Pennsylvaniav.Raymond BlizzardCASE', 'Pennsylvaniav.Shakina Thompson\nCASE INFORMATION']

In [ ]:
otn = '(?<=OTN\:\s{2})\w\s\d{6}\-\d+(?=Originating)'
otn_t = ['2004 OTN:  N 247812-5Originating',
        '1985 OTN:  Originating', # expected to "fail"
        '2001 OTN:  N 069615-0Originating',
        '1985 OTN:  M 226837-2Originating']

In [ ]:
gen_regex = lambda x: re.compile(r'%s' % x)

In [ ]:
def gen_keys():
    r = {
        'DocketNumber': (gen_regex(dn), None),
        'FiledDate': (gen_regex(fd), None),
        'DisposingJudge': (gen_regex(ja), None),
        'FullName': (gen_regex(fml), None), # needs to be split FirstName LastName MiddleName
        'OffenseTrackingNumber': (gen_regex(otn), None)
    }
    
    return r

In [ ]:
result_stats = {
    # 
}

In [ ]:
def scrape_pdf(filename):
    path = '{}/{}'.format(path_to_dockets, filename)
    data_keys = gen_keys()
    
    with open(path, 'rb') as f:
        pdf = PdfFileReader(f)
        #info = pdf.getDocumentInfo()
        page_count = pdf.getNumPages()
        
        print('File Path: {}'.format(f.name))

        for p in range(page_count):
            print('\tPage', p)
            
            page_data = pdf.getPage(p).extractText()
            # @note may have to replace `\n` with `\s`
            print(page_data)
            
            for k in data_keys:
                d = data_keys
                
                if d[k][1] is None:
                    m = re.search(d[k][0], page_data)

                    if m and m.group():
                        print('\t\t{}: {}'.format(k, m.group()))
                        d[k] = (d[k][0], m.group())
            

In [ ]:
f = scrape_pdf(dockets[13])

In [ ]:
for docket in dockets:
    try:
        scrape_pdf(docket)
    except:
      print("Something else went wrong") 